In [ ]:
# 🔢 TIOI PZ2: Классификация цифр MNIST (чёт/нечёт)

Продвинутая система машинного обучения для определения чётности цифр из датасета MNIST.

**Возможности:**
- 🧠 Нейронная сеть и логистическая регрессия
- 📊 MLflow для отслеживания экспериментов  
- 🎨 Интерактивное рисование цифр
- 📈 Детальная визуализация результатов
- 🔢 Поддержка составных цифр (двухзначные числа)


In [ ]:
!pip install matplotlib numpy scikit-learn mlflow seaborn


In [ ]:
# Импорты и основные функции
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import mlflow

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-np.clip(z, -250, 250)))

def accuracy(pred, y):
    return (pred == y).mean()

def even_odd_labels(y_digit):
    """Преобразует цифры в метки чёт/нечёт"""
    return (y_digit.astype(int) % 2).reshape(-1, 1)

def load_mnist_even_odd(test_size=0.2, seed=42):
    """Загружает MNIST и преобразует в задачу чёт/нечёт"""
    print("→ Загружаю MNIST...")
    mnist = fetch_openml('mnist_784', parser='auto')
    X = mnist['data'].to_numpy().astype(np.float32) / 255.0
    y_digits = mnist['target'].to_numpy().astype(int)
    
    y = even_odd_labels(y_digits)
    X_train, X_test, y_train, y_test, _, y_digits_test = train_test_split(
        X, y, y_digits, test_size=test_size, random_state=seed, stratify=y)
    
    print(f"  Train: {len(X_train)}, Test: {len(X_test)}")
    return X_train, X_test, y_train, y_test, y_digits_test

print("✅ Основные функции загружены!")


In [ ]:
# Модели машинного обучения
class NeuralNetwork:
    def __init__(self, n_in, n_hidden, lam=1e-4, seed=0):
        rng = np.random.default_rng(seed)
        self.params = {
            "W1": rng.normal(0, np.sqrt(2/n_in), size=(n_in, n_hidden)),
            "b1": np.zeros((1, n_hidden)),
            "W2": rng.normal(0, np.sqrt(2/n_hidden), size=(n_hidden, 1)),
            "b2": np.zeros((1, 1))
        }
        self.lam = lam
    
    def _forward(self, X):
        z1 = X @ self.params["W1"] + self.params["b1"]
        a1 = sigmoid(z1)
        z2 = a1 @ self.params["W2"] + self.params["b2"]
        a2 = sigmoid(z2)
        return a2, (X, z1, a1, z2, a2)
    
    def _backward(self, cache, y):
        X, z1, a1, z2, a2 = cache
        m = len(X)
        dz2 = a2 - y
        dW2 = a1.T @ dz2 / m + self.lam * self.params["W2"]
        db2 = dz2.mean(axis=0, keepdims=True)
        dz1 = (dz2 @ self.params["W2"].T) * a1 * (1 - a1)
        dW1 = X.T @ dz1 / m + self.lam * self.params["W1"]
        db1 = dz1.mean(axis=0, keepdims=True)
        return {"W1": dW1, "b1": db1, "W2": dW2, "b2": db2}
    
    def grads(self, X, y):
        a2, cache = self._forward(X)
        return self._backward(cache, y)
    
    def predict(self, X, batch=1024):
        out = []
        for i in range(0, len(X), batch):
            a2, _ = self._forward(X[i:i+batch])
            out.append(a2)
        return (np.vstack(out) >= 0.5).astype(int)

class GradientDescent:
    def __init__(self, lr=0.1):
        self.lr = lr
    
    def step(self, params, grads):
        for k in params:
            params[k] -= self.lr * grads[k]

print("✅ Модели определены!")


In [ ]:
# Загрузка данных и обучение модели
print("🔄 Загружаем данные MNIST...")
X_train, X_test, y_train, y_test, y_digits_test = load_mnist_even_odd()

# Создаём модель
model = NeuralNetwork(X_train.shape[1], 64, lam=1e-4)
optimizer = GradientDescent(lr=0.5)

# Обучение
epochs = 20
batch_size = 256
print(f"🧠 Обучаем нейросеть ({epochs} эпох)...")

history = {"train_acc": [], "test_acc": []}
idx = np.arange(len(X_train))

for ep in range(1, epochs+1):
    np.random.shuffle(idx)
    for start in range(0, len(X_train), batch_size):
        b = idx[start:start+batch_size]
        grads = model.grads(X_train[b], y_train[b])
        optimizer.step(model.params, grads)
    
    if ep % 5 == 0 or ep == epochs:
        train_acc = accuracy(model.predict(X_train), y_train)
        test_acc = accuracy(model.predict(X_test), y_test)
        history["train_acc"].append(train_acc)
        history["test_acc"].append(test_acc)
        print(f"Эпоха {ep:2d}: train {train_acc:.3f}, test {test_acc:.3f}")

print("✅ Обучение завершено!")


In [ ]:
# Визуализация результатов
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# График точности
epochs_shown = [5, 10, 15, 20]
ax1.plot(epochs_shown, history["train_acc"], 'bo-', label='Обучающая выборка')
ax1.plot(epochs_shown, history["test_acc"], 'ro-', label='Тестовая выборка')
ax1.set_xlabel('Эпоха')
ax1.set_ylabel('Точность')
ax1.set_title('Точность модели')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Confusion Matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
           xticklabels=["Чётная", "Нечётная"], 
           yticklabels=["Чётная", "Нечётная"], ax=ax2)
ax2.set_title('Матрица ошибок')
ax2.set_xlabel('Предсказанные метки')
ax2.set_ylabel('Истинные метки')

plt.tight_layout()
plt.show()

# Статистика
final_accuracy = accuracy(y_pred, y_test)
print(f"\n🎯 Финальные результаты:")
print(f"   Точность: {final_accuracy:.4f}")
print(f"   Чётных правильно: {cm[0,0]}/{cm[0,0]+cm[0,1]}")
print(f"   Нечётных правильно: {cm[1,1]}/{cm[1,0]+cm[1,1]}")
